# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(root, dirs, files, '\n')
    #print(file_path_list)

/home/workspace
/home/workspace/event_data [] ['2018-11-27-events.csv', '2018-11-04-events.csv', '2018-11-07-events.csv', '2018-11-09-events.csv', '2018-11-19-events.csv', '2018-11-05-events.csv', '2018-11-22-events.csv', '2018-11-16-events.csv', '2018-11-26-events.csv', '2018-11-24-events.csv', '2018-11-29-events.csv', '2018-11-15-events.csv', '2018-11-20-events.csv', '2018-11-06-events.csv', '2018-11-18-events.csv', '2018-11-21-events.csv', '2018-11-10-events.csv', '2018-11-23-events.csv', '2018-11-02-events.csv', '2018-11-28-events.csv', '2018-11-03-events.csv', '2018-11-13-events.csv', '2018-11-30-events.csv', '2018-11-12-events.csv', '2018-11-01-events.csv', '2018-11-14-events.csv', '2018-11-25-events.csv', '2018-11-08-events.csv', '2018-11-17-events.csv', '2018-11-11-events.csv'] 



#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[1])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra_db_project 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('cassandra_db_project')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# 1.Query 1
## 1.1 Drop table if needed, create & insert statements

In [8]:
# table drop statement (if needed `drop` varaible should be set to 'yes')
drop_tab_query = 'drop table if exists album'
drop = 'yes'

if drop == 'yes':
    session.execute(drop_tab_query)
    print('Table dropped')
else:
    pass

# table create statement
create_tab_query = '''
    create table if not exists album (
        sessionId INT, 
        itemInSession INT,
        artist TEXT, 
        song TEXT, 
        length FLOAT, 
        primary key (
            sessionId,
            itemInSession
        )
    )
'''

# excecution create statement
try:
    session.execute(create_tab_query)
except Exception as e:
    print(e)

# table create statement
insert_tab_query = '''
    insert into album (sessionId, itemInSession, artist, song, length)
    values (%s, %s, %s, %s, %s)
'''

Table dropped


## 1.2 Inser from `event_datafile_new.csv` into `album` table

In [9]:
# read every line from `event_datafile_new.csv`
# insert values to `album` from columns as per slicer (-3, 3, etc.)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = insert_tab_query
        session.execute(query, (int(line[-3]), int(line[3]), line[0], line[-2], float(line[5])))

## 1.3 Do a SELECT to verify that the data have been inserted into each table

In [10]:
# query statement
query = "select artist, song, length from album where sessionId = 338 and itemInSession = 4 "

# execute create statement
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# print outcomes
for i in rows:
    print(i.artist, i.song, i.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## 1.4 Check CQL results with a source file

In [11]:
# read source file into Pandas & apply filters
test = pd.read_csv(file)
test.loc[(test['sessionId'] == 338) & (test['itemInSession'] == 4), ['artist', 'song', 'length']].reset_index(drop = True)

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


# 2.Query 2
## 2.1 Drop table if needed, create & insert statements

In [12]:
# drop table if needed (drop should be yes)
drop_tab_query1 = 'drop table if exists song_played_by_user'
drop = 'yes'

if drop == 'yes':
    session.execute(drop_tab_query1)
    print('Table dropped')
else:
    pass

# table create statemant
create_tab_query1 = '''
    create table if not exists song_played_by_user (
        userId INT, 
        sessionId INT,
        itemInSession INT,
        artist TEXT,
        song TEXT,
        firstName TEXT, 
        lastName TEXT,
        primary key (
            (
                userId,
                sessionid
            ),
            itemInSession
        )
    )
'''
# excecuting create statement
try:
    session.execute(create_tab_query1)
except Exception as e:
    print(e)

# table insert statemant
insert_tab_query1 = '''
    insert into song_played_by_user (userid, sessionId, itemInSession, artist, song, firstName, lastName)
    values (%s, %s, %s, %s, %s, %s, %s)
'''           

Table dropped


## 2.2 Inser from `event_datafile_new.csv` into `song_played_by_user` table

In [13]:
# read every line from `event_datafile_new.csv`
# insert values to `album` from columns as per slicer (-3, 3, etc.)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            query = insert_tab_query1
            session.execute(query, (int(line[-1]), int(line[-3]), int(line[3]), line[0], line[-2], line[1], line[4]))
        except Exception as e:
            print(e)

## 2.3 Do a SELECT to verify that the data have been inserted into each table

In [14]:
# query statement
query = '''
    select 
        artist, song, firstName, lastName 
    from song_played_by_user 
    where userid = 10 and sessionid = 182 order by itemInSession
    '''
# execute create statement

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# print outcomes
for i in rows:
    print(i.artist, i.song, i.firstname, i.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## 2.4 Check CQL results with a source file

In [15]:
# read source file into Pandas & apply filters
(
    test
    .sort_values(by = 'itemInSession')
    .loc[(test['sessionId'] == 182) & (test['userId'] == 10), ['artist', 'song', 'firstName', 'lastName']]
    .reset_index(drop = True)
)

,artist,song,firstName,lastName
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


# 3.Query 3
## 3.1 Drop table if needed, create & insert statements

In [16]:
# drop table if needed (drop should be yes)
drop_tab_query2 = 'drop table if exists user_song_history'
drop = 'yes'

if drop == 'yes':
    session.execute(drop_tab_query2)
    print('Table dropped')
else:
    pass

# table create statemant
create_tab_query2 = '''
    create table if not exists user_song_history (
        song TEXT,
        sessionId INT,
        itemInSession INT,
        firstName TEXT, 
        lastName TEXT,
        primary key (
            song,
            sessionId,
            itemInSession
        )
    )
'''

# excecuting create statement
try:
    session.execute(create_tab_query2)
except Exception as e:
    print(e)

# table insert statemant
insert_tab_query2 = '''
    insert into user_song_history (song, sessionId, itemInSession, firstName, lastName)
    values (%s, %s, %s, %s, %s)
'''   

Table dropped


## 3.2 Inser from `event_datafile_new.csv` into `user_song_history` table

In [17]:
# read every line from `event_datafile_new.csv`
# insert values to `album` from columns as per slicer (-3, 3, etc.)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = insert_tab_query2
        try:
            session.execute(query, (line[-2], int(line[3]), int(line[-1]), line[1], line[4]))
        except Exception as e:
            print(e)

## 3.3 Do a SELECT to verify that the data have been inserted into each table

In [18]:
# query statement
query = "select firstname, lastname from user_song_history where song = 'All Hands Against His Own'"

# execute create statement
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# print outcomes
for i in rows:
    print(i.firstname, i.lastname)

Tegan Levine
Sara Johnson
Jacqueline Lynch


## 3.4 Check CQL results with a source file

In [19]:
# read source file into Pandas & apply filters
test.loc[test['song'] == 'All Hands Against His Own', ['firstName', 'lastName']].reset_index(drop = True)

,firstName,lastName
0,Tegan,Levine
1,Sara,Johnson
2,Jacqueline,Lynch


### Drop the tables before closing out the sessions

In [20]:
# loop over all tables
for i in ['album', 'song_played_by_user', 'user_song_history']:
    try:
        rows = session.execute(f'drop table if exists {i}')
        print(f'{i} - table dropped!')
    except Exception as e:
        print(e)

album - table dropped!
song_played_by_user - table dropped!
user_song_history - table dropped!


### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()